In [1]:
import nltk

In [2]:
import nltk 
from sklearn.datasets import fetch_20newsgroups
import pandas as pd 
from nltk.corpus import stopwords 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\이상은\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
# 뉴스 다운로드 및 전처리 
def get_news() : 
    # 20newsgroup 다운로드 
    dataset = fetch_20newsgroups(shuffle = True, random_state = 1, remove = ('headers', 'footers', 'quotes'))
    documents = dataset.data
    
    news_df = pd.DataFrame({'document' : documents})
    
    
    # 전처리 
    news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") # 특수문자 제거
    news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
    # 길이가 3이하인 단어는 제거
    news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower()) # 전체 단어에 대한 소문자 변환 
    tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split()) # 토큰화 
    
    stop_words = stopwords.words('english') # NLTK 불용어처리 
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stop_words]))

In [6]:
tokenized_docs = get_news()

In [7]:
tokenized_docs

0        well sure story seem biased disagree statement...
1        yeah expect people read actually accept hard a...
2        although realize principle strongest points wo...
3        notwithstanding legitimate fuss proposal much ...
4        well change scoring playoff pool unfortunately...
                               ...                        
11309    danny rubenstein israeli journalist speaking t...
11310                                                     
11311    agree home runs clemens always memorable kinda...
11312    used deskjet orange micros grappler system upd...
11313    argument murphy scared hell came last year han...
Name: clean_doc, Length: 11314, dtype: object

In [8]:
# 공백으로 토큰 분리 
def my_tokenizer(text) : 
    return text.split()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer(tokenizer = my_tokenizer)
tfidf = tfidf_vect.fit_transform(tokenized_docs)

#모델 선언 
lda = LatentDirichletAllocation(n_components = 20)
lda_output = lda.fit_transform(tfidf)

In [10]:
lda_output

array([[0.00648215, 0.00648215, 0.00648215, ..., 0.70646002, 0.00648215,
        0.00648215],
       [0.00747962, 0.00747962, 0.00747962, ..., 0.59768233, 0.00747962,
        0.00747962],
       [0.00711621, 0.00711621, 0.00711621, ..., 0.67763737, 0.00711621,
        0.00711621],
       ...,
       [0.01171825, 0.01171825, 0.37940604, ..., 0.40966547, 0.01171825,
        0.01171825],
       [0.01313165, 0.01313165, 0.01313165, ..., 0.30708252, 0.01313165,
        0.01313165],
       [0.00555874, 0.00555874, 0.00555874, ..., 0.57912157, 0.00555874,
        0.00555874]])

In [12]:
!pip install pyLDAvis

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97717 sha256=20cb396e23c03113aa67ee531fd6bd80cde2fee749d53eeed02b7ec77f861507
  Stored in directory: c:\users\이상은\appdata\local\pip\cache\wheels\57\de\11\0a038be70c2c212ce45fa0f4f9da165bb5dd87de1288394dc3
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=8542df336329de31876384644f35285b778345137ef2bc39ac7ddadb9f4fbb32
  Stored in directory: c:\users\이상은\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32047 sha256=611f7f124d8de25bed9d7b1071adf5410d3f9ffc22541da12fe7b1166cfd6bad
  Stored in directory: c:\users\이상은\appdata\local\pip\cache\wheels\5e\e0\ba\0dd7bb3f79264f8f60690da62918081bb8c3fb7442c38bfddd
Successfully built pyLDAvis future funcy


In [13]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)